In [7]:
from django.http import JsonResponse
from django.views.decorators.http import require_http_methods
from django.core.cache import cache
import requests
import os
import asyncio
import aiohttp
from django.shortcuts import render, redirect
from django.urls import reverse
from django.conf import settings
import json

BASE_URL = "https://open.api.nexon.com/maplestory/v1"
API_KEY = "test_c3a1c9dd9898748983795717d4054a73bb5713044ca4fec851806bea3c3a419aefe8d04e6d233bd35cf2fabdeb93fb0d"

# 공통적인 API 호출 로직
async def get_api_data(session, endpoint, params=None):
    headers = {"x-nxopen-api-key": API_KEY}
    url = f"{BASE_URL}{endpoint}"
    try:
        async with session.get(url, headers=headers, params=params) as response:
            response.raise_for_status()
            return await response.json()
    except aiohttp.ClientError as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None

# 캐릭터 ID 조회 함수
async def get_character_id(session, character_name):
    return await get_api_data(session, "/id", {"character_name": character_name})


# 캐릭터 정보를 가져오는 메인 함수
async def get_character_info(character_name):
    async with aiohttp.ClientSession() as session:
        character_id_data = await get_character_id(session, character_name)
        if not character_id_data or 'ocid' not in character_id_data:
            return None

        ocid = character_id_data['ocid']

        basic_info = await get_api_data(session, "/character/basic", {"ocid": ocid})
        stat_info = await get_api_data(session, "/character/stat", {"ocid": ocid})
        item_equipment_info = await get_api_data(session, "/character/item-equipment", {"ocid": ocid})
        ability_info = await get_api_data(session, "/character/ability", {"ocid": ocid})
        set_effect_info = await get_api_data(session, "/character/set-effect", {"ocid": ocid})
        link_skill_info = await get_api_data(session, "/character/link-skill", {"ocid": ocid})
        hyper_stat_info = await get_api_data(session, "/character/hyper-stat", {"ocid": ocid})
        symbol_equipment_info = await get_api_data(session, "/character/symbol-equipment", {"ocid": ocid})
        hexamatrix_stat_info = await get_api_data(session, "/character/hexamatrix-stat", {"ocid": ocid})
        popularity_info = await get_api_data(session, "/character/popularity", {"ocid": ocid})

        character_data = {
            "basic_info": basic_info,
            "stat_info": stat_info,
            "item_equipment_info": item_equipment_info,
            "ability_info": ability_info,
            "set_effect_info": set_effect_info,
            "link_skill_info": link_skill_info,
            "hyper_stat_info": hyper_stat_info,
            "symbol_equipment_info": symbol_equipment_info,
            "hexamatrix_stat_info": hexamatrix_stat_info,
            "popularity_info": popularity_info
        }

        return character_data

# 캐릭터 검색 뷰
def character_search_view(request):
    query = request.GET.get('q')
    if query:
        character_info = asyncio.run(get_character_info(query))
        return render(request, 'character_search.html', {
            'character_info': character_info,
            'query': query
        })
    return render(request, 'character_search.html')

# 캐릭터 정보 뷰
def character_info_view(request, character_name):
    character_info = asyncio.run(get_character_info(character_name))

    if character_info:
        if 'stat_info' in character_info:
            stat_info = character_info['stat_info']
            if 'final_stat' in stat_info:
                final_stats = {stat['stat_name']: stat['stat_value'] for stat in stat_info['final_stat']}
        context = {
            'character_info': character_info,
            'final_stats': final_stats if 'final_stats' in locals() else {},
        }
        return render(request, 'character_info.html', context)
    else:
        return render(request, 'error.html', {"message": "캐릭터 정보를 찾을 수 없습니다."})


In [8]:
import nest_asyncio
import asyncio
from IPython.display import display, HTML
import json

# Jupyter 환경에서 비동기 실행을 허용하기 위해 nest_asyncio 패키지를 사용
nest_asyncio.apply()

# 비동기 함수 실행
character_name = "무당햄스터"
character_info = await get_character_info(character_name)

# JSON 데이터를 보기 좋게 포매팅하고 HTML로 감싸서 출력
formatted_data = json.dumps(character_info, ensure_ascii=False, indent=4)
html_output = f"<pre style='white-space: pre-wrap; word-wrap: break-word; max-height: 500px; overflow-y: auto;'>{formatted_data}</pre>"

# 결과를 스크롤 가능한 블록으로 표시
display(HTML(html_output))
